In [1]:
import os
os.chdir('../')
#os.environ['OMOP_VOC_PATH'] = 'resources/omop_voc/20200217_ndc_rxnorm/'
os.environ['OMOP_VOC_PATH'] = 'resources/omop_voc/20200422_mesh_snomed/'

In [2]:
#https://pennchime.herokuapp.com/
#https://seaborn.pydata.org/examples/index.html

In [2]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import pytest

from rwd_analytics.cohort import CohortBuilder
#from rwd_analytics.features_selection import FeaturesSelection, time_at_risk, get_features_scores
from rwd_analytics.lookups import Descendants, Concept, ConceptRelationship, ComorbidConditions, Ingredient
from rwd_analytics.treatment_line import last_activity_date, agg_lot_by_patient, line_generation_preprocess, LinesOfTherapy
from rwd_analytics.predictions import get_matching_pairs

In [15]:
df = pd.DataFrame({
    'person_id':[1, 1, 1],
    'info':['Hello', 'Hello', 'Hello']
})
df = dd.from_pandas(df, npartitions=1).set_index('person_id')

In [18]:
#df.loc[[1, 2]].compute()
df.loc[df.index.isin([1, 2])].compute()

,info
person_id,
1,Hello
1,Hello
1,Hello


In [ ]:
def transform(df):
    df = df.drop_duplicates()
    try:
        df = df.compute()
    except:
        ''
    return df.head()

In [14]:
Concept(vocabulary_id='ICD9CM').concept

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
npartitions=3,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,int64,object
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [4]:
ConceptRelationship(relationship_id='Maps to')()

TypeError: __call__() missing 1 required positional argument: 'concept_ids'

In [ ]:
ConceptAncestor(ancestor_concept_id=['', ''])()

In [ ]:
ConceptAncestor(ancestor_concept_id=s)()

In [8]:
l = [24, 23]

In [10]:
isinstance(l, list)

True

In [13]:
isinstance(pd.Series(['hello']), pd.Series)

True

In [6]:
import pandas as pd
import dask.dataframe as dd
import psycopg2
import getpass

from rwd_analytics.lookups import Concept, ConceptRelationship

ncts = ['NCT04083235', 'NCT03945773']
connection = psycopg2.connect(user = "anthony",
                              password = getpass.getpass('Enter your password : '),
                              host = "aact-db.ctti-clinicaltrials.org",
                              port = "5432",
                              database = "aact")
df = pd.read_excel('/Volumes/GoogleDrive/My Drive/2 - Missions/IPSEN/Box/Projects/Clinical Trials References/SML_013_OncologyStudies.xlsx')
ncts = [nct for nct in df['NCT Number'].unique() if pd.notnull(nct)]
res = pd.read_sql_query("""
SELECT * FROM ctgov.browse_conditions WHERE nct_id IN (%s);
""" %(','.join(["'%s'" % (c) for c in ncts])), con=connection)

# Getting concept ID for the MeSH code
concept_mesh = Concept(vocabulary_id='MeSH').concept.compute()
df = res.merge(concept_mesh, how='left', left_on='mesh_term', right_on='concept_name')
concept_ids = df.concept_id.unique().tolist()

# Getting the standard concept ID
cr = ConceptRelationship(
    concept_id_1=concept_ids,
    relationship_id='Maps to').concept_relationship.compute()
df = df.merge(cr, how='left', left_on='concept_id', right_on='concept_id_1')
df = df[['nct_id', 'mesh_term', 'domain_id', 'concept_id_2']]
df = df.rename(columns={'concept_id_2':'concept_id'})
df = Concept(vocabulary_id='SNOMED').get_info(df, ['concept_name'])

Enter your password : ········


In [7]:
df

,nct_id,mesh_term,domain_id,concept_id,concept_name
0,NCT00080015,Lung Neoplasms,Condition,4128888.0,Neoplasm of lung
1,NCT00080015,Small Cell Lung Carcinoma,Condition,4110591.0,Small cell carcinoma of lung
2,NCT00092287,Carcinoid Tumor,Condition,40481907.0,Carcinoid tumor
3,NCT00092287,Malignant Carcinoid Syndrome,Condition,199771.0,Carcinoid syndrome
4,NCT00092287,Syndrome,Condition,4274025.0,Disease
...,...,...,...,...,...
313,NCT03995524,Stroke,Condition,381316.0,Cerebrovascular accident
314,NCT04050527,Muscle Spasticity,Condition,4329728.0,Spasticity
315,NCT04083235,Adenocarcinoma,Condition,40484156.0,Malignant adenomatous neoplasm
316,NCT04106349,Carcinoma,Condition,36716620.0,Malignant epithelial neoplasm


In [ ]:
len(df[df['concept_id'].isna()])

In [ ]:
len(df.concept_name.unique().tolist())

In [ ]:
len(df.mesh_term.unique().tolist())

In [9]:
OMOP_VOC_PATH= 'resources/omop_voc/20200422_mesh_snomed/'
concept_ancestor = dd.read_csv(OMOP_VOC_PATH+'CONCEPT_ANCESTOR.csv', sep="\t")
concept_ancestor.head()

In [12]:
concept_ancestor[concept_ancestor['descendant_concept_id']==4128888].compute()

,ancestor_concept_id,descendant_concept_id,min_levels_of_separation,max_levels_of_separation
1228168,441840,4128888,4,9
1228436,438112,4128888,5,5
1237036,4042140,4128888,3,8
1238763,4047779,4128888,5,6
1239580,4028071,4128888,4,5
1241720,4117930,4128888,4,6
1242036,4102111,4128888,3,6
1242757,4132555,4128888,4,4
1246813,4112735,4128888,1,1
1248131,4113999,4128888,4,5


,ancestor_concept_id,descendant_concept_id,min_levels_of_separation,max_levels_of_separation
0,4297090,4255266,4,7
1,4297090,4302445,3,3
2,4307750,4050748,5,5
3,4301351,4207905,4,4
4,4307750,4218301,8,9


In [ ]:
class CleaningLabResults():
    def __init__(self, df):
        self.df = df
    
    def test_distance_from_range(self, row, average, std):
        if row['value_as_number']/average < 0.01:
            if row['distance_from_range'] > row['new_distance_from_range']:
                self.i = self.i + 1
                return row['new_value_as_number']
            else:
                return row['value_as_number']
            
        else:
            return row['value_as_number']
    
    def __call__(self):
        measurement_dfs = []
        for concept in self.df['measurement_concept_id'].unique().tolist():
            df = self.df[self.df['measurement_concept_id']==concept].copy()
            df = df[df['value_as_number']!=0]
            std = df.value_as_number.std()
            high = round(df[df['range_high']!=0]['range_high'].value_counts().idxmax(), 2)
            low = round(df[df['range_low']!=0]['range_low'].value_counts().idxmax(), 2)
            std = df[df['range_high']==high].value_as_number.std()
            average = df[df['range_high']==high].value_as_number.mean()
            df['range_high'] = high
            df['range_low'] = low
            print(std)
            print(average)
            print(concept)
            print('*******')
            
            self.i = 1
            while self.i != 0:
                self.i = 0
                df['distance_from_range'] = abs(df['value_as_number']-average)
                df['new_value_as_number'] = df['value_as_number']*10
                df['new_distance_from_range'] = abs(df['new_value_as_number']-average)
                df['value_as_number'] = df.apply(self.test_distance_from_range, args=(average, std), axis=1)

            self.i = 1
            while self.i != 0:
                self.i = 0
                df['distance_from_range'] = abs(df['value_as_number']-average)
                df['new_value_as_number'] = df['value_as_number']/10
                df['new_distance_from_range'] = abs(df['new_value_as_number']-average)
                df['value_as_number'] = df.apply(self.test_distance_from_range, args=(average, std), axis=1)

            measurement_dfs.append(df)
        measurement = pd.concat(measurement_dfs)
        measurement = measurement.round({'value_as_number': 1, 'range_high': 1, 'range_low': 1})
        del measurement['distance_from_range']
        del measurement['new_value_as_number']
        del measurement['new_distance_from_range']
        return measurement

In [ ]:
def describe_lot(lot, cohort_enhanced, line, censoring_date):
    lot = lot.merge(cohort_enhanced, how='left', on='person_id')
    lot['time_to_last_activity'] = (lot['last_activity_date'] - lot['start_date']).dt.days
    lot['time_to_next_treatment'] = (lot['end_date'] - lot['start_date']).dt.days
    lot['event'] = lot['end_date'].apply(lambda x:0 if x > pd.to_datetime(censoring_date, format='%Y-%m-%d') else 1)
    lot = lot[lot['event']==1]
    lot = lot[lot['line_number']==line]
    lot = lot.groupby('regimen_name').agg({
        'person_id':['count'],
        'time_to_next_treatment':['median'],
        'time_to_last_activity':['median']
    })
    lot = lot[lot[('person_id', 'count')]>=50]
    return lot.sort_values(by=[('time_to_next_treatment', 'median')], ascending=False)

In [ ]:
# chi-squared test with similar proportions
from scipy.stats import chi2_contingency
from scipy.stats import chi2

xi_results = pd.DataFrame({
    'YY': [],
    'YN': [],
    'NY': [],
    'NN': [],
    'stat': [],
    'p': [],
    'dof': [],
    'probability': [],
    'interpret test-statistic': [],
    'interpret p-value': []
})
table = [[16, 197], [37847, 2286732]]
stat, p, dof, expected = chi2_contingency(table)
prob = 0.95
critical = chi2.ppf(prob, dof)

# interpret test-statistic
if abs(stat) >= critical:
    test_statistic = 'Dependent (reject H0)'
else:
    test_statistic = 'Independent (fail to reject H0)'

# interpret p-value
alpha = 1.0 - prob
if p <= alpha:
    p_value = 'Dependent (reject H0)'
else:
    p_value = 'Independent (fail to reject H0)'

# contingency table
xi_add_results = pd.DataFrame({
    'YY': [table[0][0]],
    'YN': [table[0][1]],
    'NY': [table[1][0]],
    'NN': [table[1][1]],
    'stat': [stat],
    'p': [p],
    'dof': [dof],
    'probability':[prob],
    'interpret test-statistic': [test_statistic],
    'interpret p-value': [p_value]
})

xi_results = xi_results.append(xi_add_results)

In [ ]:
xi_results

In [ ]:
test_comorbid_conditions()

In [ ]:
test_concept_info()

In [ ]:
What is the time for defining a gap?


In [ ]:
era

In [ ]:
era = t.groupby(['person_id', 'concept_id']).agg({
    'start_date':['min', 'max', 'count'],
    'gap':['cumsum']
})
era['era_duration'] = (era['max'] - era['min']).dt.days
era = era.reset_index()

In [ ]:
t

In [ ]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import math

# sudo docker-compose exec --user root  notebook bash
# pip install -U scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, log_loss, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [ ]:
df = pd.DataFrame({
    'age_at_index':[18, 28, 8, 48, 58],
    'gender = female':[1, 0, 1, 0, 0],
    'condition_1':[1, 1, 1, 0, 0],
    'condition_2':[1, 1, 1, 0, 0],
    'target':[0, 1, 1, 0, 0]
})

In [ ]:
get_features_scores(df, 4)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = df.iloc[:,0:4]
scaler = StandardScaler().fit(X)
standardized_X = scaler.transform(X)

from sklearn.linear_model import LogisticRegression
#X = df.iloc[:,0:4]  #independent columns
X = standardized_X
y = df.iloc[:,-1]    #target column i.e price range
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
clf = LogisticRegression(random_state=0, solver='lbfgs').fit(X, y)
#clf.predict(X)
proba = pd.DataFrame(clf.predict_proba(X))[[1]]
proba.columns = ['probability']
proba['probability'] = proba['probability'].apply(lambda x:round(x, 4))
pd.concat([df, proba], axis=1)

In [ ]:
X

In [ ]:
clf.score(X, y)

In [ ]:
import pandas as pd
import dask.dataframe as dd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from rwd_analytics.lookups import Descendants, ComorbidConditions

def get_features_scores(df, n_features):
    X = df.iloc[:,0:n_features]  #independent columns
    y = df.iloc[:,-1]    #target column i.e price range
    
    #apply SelectKBest class to extract top 10 best features
    bestfeatures = SelectKBest(score_func=chi2, k=n_features)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns, dfscores], axis=1)

    # naming the dataframe columns and rounding results
    featureScores.columns = ['Specs', 'Score']
    featureScores['Score'] = featureScores['Score'].round(2)
    return featureScores.nlargest(n_features, 'Score')

In [ ]:
get_features_scores(df, 4)

In [ ]:
PredictionModels(df)()

In [ ]:
class PredictionModels():
    def __init__(self, df):
        a = df[df['target']==1]
        b = df[df['target']==0]
        print('Subjects in target=1: '+str(len(a)))
        print('Subjects in target=0: '+str(len(b)))
        
        X = df.drop('target', axis=1)
        y = df['target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        #if (len(a)+len(b))/20 < abs(len(b)-len(a)):
        #    print('Imbalanced dataset: resampling train set')
        #    print('************************************')
        #    c = math.trunc((max(len(a), len(b)) + min(len(a), len(b)))/2)
        #    a = self.X_train.sample(n=c, replace=True, random_state=3)
        #    b = self.y_train.sample(n=c, replace=True, random_state=3)
        #    df = pd.concat([a, b], ignore_index=True)

    def __call__(self):
        feedback = pd.DataFrame(columns=['Classifier', 'Model Score', 'Accuracy Score'])
        classifiers = [
            DummyClassifier(strategy='most_frequent', random_state=0),
            KNeighborsClassifier(3),
            SVC(kernel="rbf", C=0.025, probability=True),
            #NuSVC(probability=True),
            DecisionTreeClassifier(),
            RandomForestClassifier(),
            AdaBoostClassifier(),
            GradientBoostingClassifier()
            ]

        for classifier in classifiers:
            model = classifier.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)
            feedback_temp = pd.DataFrame({
                'Classifier':[classifier],
                'Model Score':[model.score(self.X_train, self.y_train)],
                'Accuracy Score':[accuracy_score(self.y_test, y_pred)]
            })
            feedback.append(feedback_temp)
            print(classifier)
            print("Training score: %.3f" % model.score(self.X_train, self.y_train))
            print("Test score: %.3f" % accuracy_score(self.y_test, y_pred))
            print('*** Confusion matrix ***')
            print(confusion_matrix(self.y_test, y_pred))
            print('*** Classification report ***')
            print(classification_report(self.y_test, y_pred))
            print('************************************')
        
        return feedback